### Prompt Engineering MENTAL HEALTH SERVICES ###

In [46]:
import os
import pandas as pd
import numpy as np
import logging

logger = logging.getLogger(__name__)

# Import this module with autoreload
%load_ext autoreload
%autoreload 2
import llmt
from llmt.llmtools import Prompt
from llmt.llmtools import process_prompt
from llmt.openai import OpenAI, OpenAIModel, create_messages
from llmt.openai import MentalHealth
from llmt.performance import Performance

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [47]:
# Parameters
model = 'gpt-4o'
temperature = 0

# Directories and files
data_dir = os.path.join(os.environ.get('HOME'), 'home_data', 'hcp')
test_file_name = 'hcp-alldata-250413.parquet'
test_file = os.path.join(data_dir, test_file_name)
df = pd.read_parquet(test_file)
# Filter the labeled data
df = df.loc[df['dset'] == 'train'].\
                reset_index(drop=True).\
                astype({'mental_health': int,
                        'inpatient': int,
                        'outpatient': int})
display(df.head())
print(df.shape)

,id,name,description,mental_health,inpatient,outpatient,dset
0,431643-07,Actriv,Provider of healthcare staffing services based...,2,0,0,train
1,310749-31,Alima,Operator of a non-governmental organization in...,0,0,0,train
2,162054-28,Apothecare,Provider of pharmacy services intended to prov...,0,0,0,train
3,597285-28,April Health (Clinics/Outpatient Services),Provider of mental health services intended to...,1,0,1,train
4,373978-90,Arise Child and Family Service,Operator of independent living centers caterin...,2,0,0,train


(187, 7)


### Prompt development ###

In [50]:
# Pick an example of a company that provides mental health services
mental_health = 1
inpatient = 1
id_list = list(df.loc[(df['mental_health'] == mental_health) & (df['inpatient'] == inpatient), 'id'].unique())
print(f'Found {len(id_list)} company ids')

# Pick a company ID
idx = 10
company_id = id_list[idx]
# company_id = '135432-64'
ser_id = df.loc[df['id'] == company_id].iloc[0]
display(ser_id)
name = ser_id['name']
description = ser_id['description']
print()
print(name)
print(description)

Found 62 company ids


id                                                       444148-30
name                                        Purpose Healing Center
description      Provider of alcohol and drug treatment centers...
mental_health                                                    1
inpatient                                                        1
outpatient                                                       1
dset                                                         train
Name: 116, dtype: object


Purpose Healing Center
Provider of alcohol and drug treatment centers intended for inpatient and outpatient programs. The company offers a variety of programs including medical detox, medication assisted treatment, and inpatient and outpatient rehab, patients, including dual diagnosis, mental health services, enabling patients to get a safe recovery environment away from drugs.


In [51]:
variable = 'mental_health'
prompt_version = 3
prompt_name = f'{variable}_system_{str(prompt_version).zfill(2)}'
system_prompt = Prompt().load(prompt_name=prompt_name)
print(prompt_name)

# For the user prompt, we want to include the name of the business and the description
def create_user_prompt(name: str, description: str):
    user_prompt = f"""
        The business {name} is described as: {description} 
        Does the business provide mental or behavioral health care services to human patients?
        """
    return process_prompt(user_prompt)

user_prompt = create_user_prompt(name=name, description=description)
messages = create_messages(system_prompt=system_prompt, user_prompt=user_prompt)

mental_health_system_03


In [52]:
# Use the OpenAIModel class
variable = 'mental_health'
pred_col = 'pred_mh'
version = prompt_version
mh_prompt_name = f'{variable}_system_{str(version).zfill(2)}'
system_prompt = Prompt().load(prompt_name=mh_prompt_name)
user_prompt = process_prompt(f"""
                The organization {name} is described as: {description} 
                Does this organization provide {variable} healthcare services?
                """)
messages = create_messages(system_prompt=system_prompt, user_prompt=user_prompt)

In [53]:
print(user_prompt)

The organization Purpose Healing Center is described as: Provider of alcohol and drug treatment centers intended for inpatient and outpatient programs. The company offers a variety of programs including medical detox, medication assisted treatment, and inpatient and outpatient rehab, patients, including dual diagnosis, mental health services, enabling patients to get a safe recovery environment away from drugs. 
Does this organization provide mental health healthcare services?


In [54]:
# Send prompt to model
model = 'gpt-4o'
temperature = 0
response_format = MentalHealth
client = OpenAI().create_client()

output = OpenAI().send_messages(messages=messages, 
                                model=model, 
                                response_format=response_format,
                                temperature=0,
                                client=client)
print(output)

{'pred_mh': True, 'pred_mh_score': 0.95, 'refusal': None}


In [55]:
# Use the class methods
model = OpenAIModel(model='gpt-4o')
response_mh = model.predict_mh(name=name, description=description, version=3, temperature=temperature)

In [56]:
print(response_mh)

{'pred_mh': 1}


In [ ]:
# Run the prompt on all data
company_id_list = sorted(list(df['id'].unique()))
results_df_list = []

for c, company_id in enumerate(company_id_list):
    if (c + 1) % 20 == 0:
        print(f'Sending description {c + 1} / {len(company_id_list)} to the model')
    df_id = df.loc[df['id'] == company_id]
    user_prompt_id = create_user_prompt(name=df_id['name'].values[0], 
                                        description=df_id['description'].values[0])
    messages = create_messages(system_prompt=system_prompt, user_prompt=user_prompt_id)
    output = OpenAI().send_messages(messages=messages,
                                    model=model,
                                    temperature=temperature,
                                    response_format=response_format,
                                    client=client)
    
   # Replace the boolean with binary outcome prediction
    key = 'pred_mh'
    output.update({key: 1 if output.get(key) == True else 0})
    results_df_list.append(df_id.assign(**output))

results_df = pd.concat(results_df_list, axis=0, ignore_index=True)
# Save the results
results_file_name = f'{variable}_{str(prompt_version).zfill(2)}_results.parquet'
results_file = os.path.join(data_dir, results_file_name)
results_df.to_parquet(results_file)
print(results_file)

In [ ]:
display(Performance(data=results_df).\
        binary_performance(true_col='mental_health', pred_col='pred_mh'))